# 10. QA with Phrase Retrieval

## 강의소개

- 10강에서는 phrase retrieval을 활용한 real-time oepn-domain question answering에 대해 배워보겠습니다.
- Phrase 단위로 retrieval한다는 것이 무엇인지 알아보고, phrase retrieval을 수행하는 방법에 대해 알아보겠습니다.
- 마지막으로, 기존 retrievar-reader 구조와 비교해 어떤 장점과 단점을 가지고 있는지 짚어볼 예정입니다.

<br>

## Further Reading

- [Real-Time Open-Domain Question Answering with Dense-Sparse Phrase Index](https://arxiv.org/abs/1906.05807)
- [Contextualized Sparse Representations for Real-Time Open-Domain Question Answering](https://arxiv.org/abs/1911.02896)

<br>

## 10.1 Phrase Retrieval in Open-Domain Question Answering

- Current Limitation of Retriever-Reader approach
- Phrase Indexing
- Query-Agnostic Decomposition

<br>

### 10.1.1 Current limitation of Retriever-Reader approach

- 1~8강 까지의 방법론은 아래와 같은 문제점을 갖게 된다.

1. Error Propagation
  - 5-10개의 문서만 reader에게 전달됨
  - 이 문서들이 올바른 문서라는 보장이 없다.
2. Query-dependent encoding
  - query 에 따라 정답이 되는 answer span 에 대한 encoding 이 달라짐
  - 상당히 비효율적이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1u0ai4M8lKUSJYMgLqiT0ufP5cry_X3nx' width=600/>

<br>

### 10.1.2 How does Document Search work?

- 위와 같은 문제점을 해결하고자 제안된 것이 phrase search 이다.
- phrase search는 document search와 상당히 유사하다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1u1Ip7L9P2jJH-riK1H5X3yBVsUNqdtAy' width=600/>

- 문서를 가져오려고 할 때 문서에 대한 sparse 또는 dense embedding vector를 만들고 새로운 question이 들어오면 관련 있는 문서를 찾기 위해 nearest neighbor search나 MIPS와 같은 방법으로 문서를 찾았다.

<br>

- Key Idea
  - Retrieve-Read 두 단계 말고 <mark>정답을 바로 search</mark>할 수는 없을까?

<br>

### 10.1.3 One solution: Phrase Indexing

- 아래와 같은 문서가 있다고 하자.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uBQw3eD0EZQAyEEY1apZVStZzWGP0BEj' width=800/>


- 이 문서에 존재하는 모든 phrase들을 모두 enumerate(열거)한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uEqpB6TPjg8dpe0iTG5_rNa_HNSg7hK5' width=800/>

- 각 phrase에 대한 임베딩을 그림의 오른쪽과 같이 맵핑해준다.
- 이 벡터들을 key vector로 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uQ9_uTSpWruhRjwe0DfOw7Ag9sukhE0t' width=800/>

- 질문이 들어왔을 때 질문 또한 임베딩 벡터로 만들어준다.
- 이 벡터는 query vector로 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uR-Y8HZJShLm7qZ7JEOqfwCPcGbS90dx' width=800/>

- 질문에 대한 query vector와 지문의 각 phrase에 대한 key vector들 중 가장 가까운 벡터를 찾는 문제로 치환할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uRwsJvlK1e9crymqd8j-AEoGEG_UyPSd' width=800/>

- 이렇게 하면 새로운 질문이 들어온다 하더라도 key vector에 대한 업데이트 없이 새로운 query vector만 계산을 해서 key vector에 연결해주는 방법을 취할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uWGK3NiVcbIBLMWLgiXxE4fjIIg6KeF0' width=800/>

<br>

### 10.1.4 Query-Agnostic Decomposition

- 위와 같은 방법으로 접근하게 됐을 때를 생각해보자.

<br>

- 기존의 MRC 모델 같은 경우에는 scoring function $F$를 구하는 문제로 볼 수 있다.
  - $F$에는 phrase, question, document가 입력으로 들어온다.
  - 이 3가지를 모델에 넣어줌으로서 scoring을 해주는 구조이다.
- 이러한 방법의 가장 큰 문제는 새로운 질문이 들어올 때마다 $a$와 $d$는 똑같다 하더라도 함수 $F$를 다시 한번 계산해줘야 한다는 점이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uXjxNp7UPWhUv2367wsO57Z8WTFHpzWn' width=600/>

- 이러한 불편함을 phrase retriever 방법으로 접근하게 되면 $G$와 $H$라는 새로운 인코더 함수를 새로 정의하는 것이 된다.
  - $G$: question을 인코딩하여 벡터 공간에 맵핑해주는 역할
  - $H$: 각 answer candidate과 문서를 벡터 공간에 맵핑해주는 역할
- $G$와 $H$를 inner product 해줘서 값이 가장 높은 $a$를 찾는 방식으로 문제를 치환해줄 수 있다.
- 이렇게 문제를 치환하는 것이 phrase retriever로 치환해주는 것과 동일한 것이 된다.
- 이렇게 해줬을 때의 장점은 $H$ 함수에 입력으로 들어가는 문서들 질문이 들어올 때마다 다시한번 인코딩 해줄 필요가 없다는 점이다.
- 하지만 이 방법의 가장 큰 단점은 $F$라는 함수가 정의됐을 때 $F$가 $G$와 $H$로 나눠질 수 있다는 가정 자체가 틀릴 수 있다는 점이다.
  - $F$가 $G$와 $H$로 나눠질 수 있다는 보장이 없다.
- 실제로는 $F$를 $G$와 $H$로 정확하게 decompose하기 보다는 $G$와 $H$를 학습을 하되 $F$에 approximate 하게 학습을 하게 된다.

- 이렇게 했을 때의 key challenge는 "어떻게 각 phrase를 vector space 상에 잘 mapping 할 수 있을까?"가 된다.
- 이에 대한 솔루션 중 하나는 Dense와 Sparse 임베딩을 둘 다 사용해보는 것이 있을 수 있다.
  - 최근에는 dense 임베딩만을 통해서 잘 되는 연구 결과들도 나오고 있다.
  - 유일한 솔루션이라고 하기 보다는 이러한 솔루션이 있다고 생각하면 될 것 같다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1u_VJbWx_CBXH9HzPPOMClJB79da5r4-8' width=600/>

<br>

## 10.2 Dense-sparse Representation for Phrases

- Dense vectors vs. Sparse vectors
- Phrase and Question Embedding
- Dense Representation
- Sparse Representation
- Scalability Challenge

<br>

### 10.2.1 Dense vectors vs Sparse vectors

- Dense와 Sparse의 각각의 장점은 명확하게 존재한다.

<br>

- Dense vectors
  - 통사적, 의미적 정보를 담는 데 효과적
- Sparse vectors
  - 어휘적 정보를 담는 데 효과적

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ucOoaoFYJj-CfO1Fdf-DVwnAxupZ9iNC' width=800/>

<br>

### 10.2.2 Phrase and Question Embedding

- Dense vector와 sparse vector를 모두 사용하여 phrase (and question) embedding
- 각 phrase에 대한 임베딩을 구할 때 Dense와 Sparse를 모두 계산한 다음에 두 벡터를 하나로  합쳐서 하나의 벡터로 고려를 하고 inner product나 nearest neighbor search를 진행한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ucjbVomZtQkIMU6ubOGNX4jbx-xSKZRK' width=600/>

- 그렇다면 각각의 dense, sparse 벡터는 어떻게 계산을 해야 할까?

<br>

### 10.2.3 Dense representation

#### 10.2.3.1 Dense vector를 만드는 방법

- 지문에서 정답에 해당하는 phrase vector를 만드는 방법은 시작점에 있는 벡터와 끝점에 있는 벡터를 합쳐주는 것이다.
- Pre-trained LM (e.g. BERT)를 이용
- Start vector와 end vector를 재사용해서 메모리 사용량을 줄임

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uifROgQb6GU50rgC0z4Lm0Uuar2x8m3h' width=600/>

- 아래 그림과 같이 "the American Library Association"에 대한 phrase vector를 구하고 싶을 때 "the"에 대한 dense vector의 일부분과 "Association"에 대한 dense vector의 일부분을 가져와서 두개의 벡터를 concat한다. 

<br>

#### 10.2.3.2 Coherency(일관성) vector

- phrase가 한 단위의 문장 구성 요소에 해당하는지를 나타냄
- 구(句)를 형성하지 않는 phrase를 걸러내기 위해 사용함
- Start vector와 end vector를 이용하여 계산

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ujvBb-KFllGYzj-pvx16GoB1faaiB0_5' width=600/>

- 위에서 "the"의 dense vector 일부분과 "Association"의 dense vector 일부분을 결합한 phrase vector에 Coherency vector를 붙여줘서 최종 phrase vector를 만든다.
  - Coherency vector는 해당 phrase vector가 좋은 벡터인지 아닌지에 대한 정보(정답인지 아닌지)를 전달해주는 역할을 수행한다.

<br>

#### 10.2.3.3 Question embedding

- Question을 임베딩할 때는 `[CLS]` 토큰 (BERT)을 활용하여 questiono vector를 만들어준다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uoTQ_FapGJ1taDAzMB0fPRJzyrHyHpnY' width=600/>

<br>

### 10.2.4 Sparse representation

#### 10.2.4.1 Sparse vector를 만드는 방법

- 문맥화된 임베딩(contextualized embedding)을 활용하여 가장 관련성이 높은 n-gram으로 sparse vector 구성

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uqbkbVF6mhX-3k4veNwGdMry5Isapql_' width=600/>

- Target으로 하고 있는 phrase에 주변에 있는 단어들과 유사성을 계산하여 그 유사성을 각 단어에 해당하는 sparse vector의 dimension에 넣어줌으로서 sparse vector를 만들 수 있다.
- TF-IDF와 유사하지만 각 문장 및 phrase마다 weight가 다이나믹하게 변하는 형태의 벡터로 만들어줄 수 있다는 차이가 있다.
- 또한 n-gram을 활용하여 주변 단어의 정보를 포함할 수 있다.

<br>

### 10.2.5 Scalability Challenge

- 위와 같은 방법으로 접근했을 때 다음과 같은 문제가 발생할 수 있다.

<br>

- In Wikipedia에 60 billion개의 phrases가 존재
- Storage, indexing, search의 scalability가 고려되어야 함

<br>

- Storage
  - Pointer, filter, scalar quantization 활용
  - 240T storage (optimization 없이 저장한 경우)
  - 1.4T storage (여러 가지 트릭을 통해 줄여서 저장한 경우)
- Search
  - FAISS를 활용해 dense vector에 대한 search를 먼저 수행 후 sparse vector로 reranking
  - 단 FAISS는 dense vector에 대한 검색만 가능하다. (sparse에 대한 검색은 불가능하다.)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1urvSxfVBmJhOOG9Syu_u1VzaP0BQkLzV' width=500/>

<br>

## 10.3 Experiment Results & Analysis

- Experiment Results 
- Limitation in Phrase Retrieval Approach
- Improving Sparse Representation - Contextualized Sparse Representation (SPARC)

<br>

### 10.3.1 Experiment Results - SQuAD-open

- SQuAD-open (Open-domain QA)
  - s/Q: seconds per query on CPU
  - #D/Q: number of ducuments visited per query
- DrQA(Retriever-reader)보다 +3.6% 성능
- 68x 빠른 inference speed (less than 1s)


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uwRUCKE2p-r6hesx_w2ATY22kf_9B53h' width=400/>

<br>

### 10.3.2 Experiment Results

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uwpO_1NhcWtsUxTRJkGpqcs3QcPD4SoN' width=600/>

<br>

### 10.3.3 Limitation in Phrase Retrieval Approach

- 다음과 같은 한계점들이 존재한다.

1) Large storage required

- 2TB SSD for phrases
- 많은 양의 RAM과 Storage를 필요로 함

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1v-pyUaMNET8OU3shKcNES9pOSXrSJfE1' width=400/>

<br>

2) 최신 Retriever-reader 모델들 대비 낮은 성능

- 그 때 당시에는 비교했던 모델들에 비해 성능이 comparable했지만 그 이후에 등장했던 모델들에 비해서는 상당히 낮은 성능을 보여주고 있다.
- SQuAD-open 결과

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uwzugbUx-yPP6nSedeBFMlbDViKt972R' width=400/>

- Natural Questions (NQ) 에서 낮은 성능
  -  Decomposability gap이 하나의 원인

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1v0ln-cShdEaink__Y4oALYUiH2iLAJTp' width=600/>

<br>

### 10.3.4 Decomposability gap

- 기존
  - Question, passage, answer가 모두 함께 encoding
- Phrase retrieval
  - Question과 passage/answer이 각각 encoding
  - Question과 passage사이 attention 정보 x
- Decomposability gap
  - $F$를 정확하게 $G$와 $H$로 분해할 수 없다는 점에서 발생하는 gap

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1v8JFbl7zhSR3q4G3hRWa4ggkqSwbxRYM' width=800/>